In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

In [2]:
# Read the train.csv into a DataFrame
data = pd.read_csv('data\Doceree-HCP_Train.csv', encoding='latin-1')
data.head()


,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
0,1001,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0
2,1003,Desktop,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,New Meadows,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
3,1004,Desktop,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,NaN,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0
4,1005,Mobile,7,174.202.231.99,a17e25be-532d-4cf5-b916-9308c8c3961f,Houston,77008.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal,Website,https://www.cureus.com/articles/58184-a-review...,Critical Care|Emergency Medicine|General Pract...,NaN,0.0


In [3]:
data = data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

data['ID'] = data['ID'].astype(str)

data = data.dropna(subset=['DEVICETYPE', 'PLATFORM_ID'])

data['BIDREQUESTIP'] = data['BIDREQUESTIP'].astype(str).apply(lambda x: x if pd.Series(x).str.contains('^(\d{1,3}\.){3}\d{1,3}$').all() else None)

data['USERZIPCODE'] = data['USERZIPCODE'].astype(str).str.replace(r'\D+', '')

data[['BROWSER', 'OS']] = data['USERAGENT'].astype(str).str.split('(', n=1, expand=True)

data['BROWSER'] = data['BROWSER'].astype(str).str.strip()

data['OS'] = data['OS'].astype(str).str.rstrip(')')

data['KEYWORDS'] = data['KEYWORDS'].apply(lambda x: '|'.join(set(x.split('|'))) if pd.notnull(x) else x)

data['IS_HCP'].fillna(0, inplace=True)

data['IS_HCP'] = data['IS_HCP'].astype(int)

print(data.head(3))

C:\Users\VANI SETH\AppData\Local\Temp\ipykernel_12316\1204343258.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['BIDREQUESTIP'] = data['BIDREQUESTIP'].astype(str).apply(lambda x: x if pd.Series(x).str.contains('^(\d{1,3}\.){3}\d{1,3}$').all() else None)


     ID DEVICETYPE  PLATFORM_ID   BIDREQUESTIP  \
0  1001    Desktop            2   170.173.0.22   
1  1002    Desktop            2  65.216.253.25   
2  1003    Desktop            2   66.232.79.22   

                        USERPLATFORMUID     USERCITY USERZIPCODE  \
0  6974dcaa-f932-480e-9fb5-c52e20e1393a     Portland     97206.0   
1  c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf    Arlington     22202.0   
2  a698de4b-e200-46dd-b5fb-40402175ae18  New Meadows     83654.0   

                                           USERAGENT            PLATFORMTYPE  \
0  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...  Online Medical Journal   
1  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...  Online Medical Journal   
2  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...  Online Medical Journal   

  CHANNELTYPE                                                URL  \
0     Website  https://www.cancertherapyadvisor.com/home/canc...   
1     Website  https://www.cancertherapyadvisor.com/home/deci...   
2

In [4]:
data.dtypes.count()

16

In [5]:
# Read the data from CSV file
test_data = pd.read_csv('data\Doceree-HCP-Test.csv', encoding='latin-1')

# Remove leading/trailing whitespaces from all string columns
test_data = test_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Convert ID column to string
test_data['ID'] = test_data['ID'].astype(str)

# Remove rows with missing values in specific columns
test_data = test_data.dropna(subset=['DEVICETYPE', 'PLATFORM_ID'])

# Clean the BIDREQUESTIP column by removing invalid IP addresses
test_data['BIDREQUESTIP'] = test_data['BIDREQUESTIP'].astype(str).apply(lambda x: x if pd.Series(x).str.contains('^(\d{1,3}\.){3}\d{1,3}$').all() else None)

# Clean the USERZIPCODE column by removing non-numeric characters
test_data['USERZIPCODE'] = test_data['USERZIPCODE'].astype(str).str.replace(r'\D+', '')

# Split the USERAGENT column into separate browser and operating system columns
test_data[['BROWSER', 'OS']] = test_data['USERAGENT'].astype(str).str.split('(', n=1, expand=True)
test_data['BROWSER'] = test_data['BROWSER'].astype(str).str.strip()
test_data['OS'] = test_data['OS'].astype(str).str.rstrip(')')

# Remove duplicates from KEYWORDS column
test_data['KEYWORDS'] = test_data['KEYWORDS'].apply(lambda x: '|'.join(set(x.split('|'))) if pd.notnull(x) else x)

# Print the cleaned data
print(test_data.head(3))


C:\Users\VANI SETH\AppData\Local\Temp\ipykernel_12316\1607418820.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  test_data['BIDREQUESTIP'] = test_data['BIDREQUESTIP'].astype(str).apply(lambda x: x if pd.Series(x).str.contains('^(\d{1,3}\.){3}\d{1,3}$').all() else None)


       ID DEVICETYPE  PLATFORM_ID     BIDREQUESTIP  \
0  115501    Desktop          2.0   75.189.231.103   
1  115502     Mobile          2.0    24.101.33.158   
2  115503    Desktop          2.0  172.118.216.142   

                        USERPLATFORMUID       USERCITY USERZIPCODE  \
0  0d5041ff-f0b6-4d1a-9ad7-0a29f7d485b4   Fayetteville     28305.0   
1  c8396dd0-969f-4d99-a40b-b7bb1f516154  Conneaut Lake     16316.0   
2  3c97a081-6518-43f8-9f26-369759cfb471         Covina     91724.0   

                                           USERAGENT            PLATFORMTYPE  \
0  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...  Online Medical Journal   
1  Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...  Online Medical Journal   
2  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...  Online Medical Journal   

  CHANNELTYPE                                                URL  \
0     Website  https://www.clinicaladvisor.com/home/features/...   
1     Website  https://www.ophthalmologyadvis

In [6]:
test_data.dtypes.count()

14

In [7]:
test_data.dtypes.describe

<bound method NDFrame.describe of ID                  object
DEVICETYPE          object
PLATFORM_ID        float64
BIDREQUESTIP        object
USERPLATFORMUID     object
USERCITY            object
USERZIPCODE         object
USERAGENT           object
PLATFORMTYPE        object
CHANNELTYPE         object
URL                 object
KEYWORDS            object
BROWSER             object
OS                  object
dtype: object>

In [8]:
# encode all the columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
test_data = test_data.apply(le.fit_transform)

In [9]:
test_data.head(3)
print(test_data.dtypes.count())

14


In [10]:
# Divide the data target variable and features
X = data.drop(['IS_HCP'], axis=1)
y = data['IS_HCP']

In [11]:
# encode all the columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X = X.apply(le.fit_transform)

In [12]:
X.dtypes.count()

15

In [13]:
X.head()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,BROWSER,OS
0,110,0,1,6580,22183,3177,10913,2857,3,0,666,692,207,10,2857
1,221,0,1,19253,40576,130,1900,2672,3,0,708,627,207,10,2671
2,332,0,1,19704,34968,2701,9156,2672,3,0,666,692,207,10,2671
3,443,0,2,4193,14676,4420,1959,2945,3,0,327,738,149,10,2945
4,554,1,6,10056,33844,1767,8359,4049,3,0,1631,418,207,10,4057


In [14]:
#scale the params
scaler = MinMaxScaler()
data['USERZIPCODE_NORMALIZED'] = scaler.fit_transform(data['USERZIPCODE'].values.reshape(-1, 1))

In [15]:
X = data[['DEVICETYPE_ENCODED', 'PLATFORMTYPE_ENCODED', 'CHANNELTYPE_ENCODED', 'URL_LENGTH', 'HAS_KEYWORD', 'USERZIPCODE_NORMALIZED']]
y = data[['IS_HCP', 'TAXONOMY']]


KeyError: "['DEVICETYPE_ENCODED', 'PLATFORMTYPE_ENCODED', 'CHANNELTYPE_ENCODED', 'URL_LENGTH', 'HAS_KEYWORD'] not in index"

In [16]:
y = pd.get_dummies(y, columns=['TAXONOMY'])

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32, callbacks=[early_stopping])

Epoch 1/10
2849/2849 [==============================] - 6s 2ms/step - loss: 63.6931 - accuracy: 0.6593 - val_loss: 0.6105 - val_accuracy: 0.7005
Epoch 2/10
2849/2849 [==============================] - 6s 2ms/step - loss: 0.6488 - accuracy: 0.6991 - val_loss: 0.6104 - val_accuracy: 0.7005
Epoch 3/10
2849/2849 [==============================] - 6s 2ms/step - loss: 0.6263 - accuracy: 0.6998 - val_loss: 0.6106 - val_accuracy: 0.7005
Epoch 4/10
2849/2849 [==============================] - 5s 2ms/step - loss: 0.6296 - accuracy: 0.6998 - val_loss: 0.6105 - val_accuracy: 0.7005
Epoch 5/10
2849/2849 [==============================] - 7s 2ms/step - loss: 0.6363 - accuracy: 0.6999 - val_loss: 0.6105 - val_accuracy: 0.7005
Epoch 6/10
2849/2849 [==============================] - 5s 2ms/step - loss: 0.6134 - accuracy: 0.6999 - val_loss: 0.6105 - val_accuracy: 0.7005
Epoch 7/10
2849/2849 [==============================] - 5s 2ms/step - loss: 0.6126 - accuracy: 0.6998 - val_loss: 0.6104 - val_accuracy

In [20]:
loss, accuracy = model.evaluate(X_val, y_val)
print('Validation loss: %.2f' % (loss*100))
print('Validation accuracy: %.2f' % (accuracy*100))

713/713 [==============================] - 1s 1ms/step - loss: 0.6104 - accuracy: 0.7005
Validation loss: 61.04
Validation accuracy: 70.05


In [21]:
model.save('model2.h5')